# Lesson 2.1 - Experiment Tracking with MLflow / W&B

***Key Concepts:*** *Experiment Trackers, MLflow, Weights & Biases*

When training your own models, you can easily get overwhelmed by the large number of different experiments you conduct and you might find yourself losing track of how different hyperparameters affected your model performance, or what exact configuration produced the best model you had trained. That is why experiment tracking tools like Tensorboard, Weights & Biases, or MLflow are often one of the first touchpoints you will have with MLOps as you progress through your ML journey.

In this lesson, we will learn how to effectively track experiments with MLflow, which is the most popular open-source experiment tracking tool.
For those of you already familiar with experiment tracking tools, we also have a short bonus section at the end on how to use Weights & Biases instead of MLflow in your ZenML pipelines.

To get started, run the following commands to install both tools with their respective dependencies.

In [ ]:
!zenml integration install mlflow -f
!zenml integration install wandb -f

Next, let's import our pipeline definition and some of the pipeline steps that we built in the previous lessons:

In [ ]:
from src.steps.evaluator import evaluator
from src.steps.importer import importer
from src.pipelines.digits_pipeline import digits_pipeline

## MLflow Experiment Tracking

[MLflow](https://mlflow.org/) is an amazing open-source MLOps platform that provides powerful tools to handle various ML lifecycle steps, such as experiment tracking, code packaging, model deployment, and more. In this lesson, we will focus on the [MLflow Tracking component](https://mlflow.org/docs/latest/tracking.html), but we will learn about other MLflow components in later lessons.

To integrate the MLFlow experiment tracker into our previously defined ZenML pipeline, all we need to adjust is the `svc_trainer` step. To do so, we define a new step `svc_trainer_mlflow` in which we use MLflow's `mlflow.sklearn.autolog()` feature to automatically log all relevant attributes of our model to MLflow. By adding an `@enable_mlflow` decorator on top of the function, ZenML then automatically initializes MLflow and takes care of the rest for us.

The following function creates such a step, parametrized by the SVC hyperparameter `gamma`, then returns a corresponding ML pipeline.

In [ ]:
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
import mlflow
from zenml.integrations.mlflow.mlflow_step_decorator import enable_mlflow
from zenml.steps import step, BaseStepConfig


def build_svc_mlflow_pipeline(gamma=1e-3):
    @enable_mlflow  # setup MLflow
    @step(
        enable_cache=False
    )  # disable caching so we log **every** run to MLflow
    def svc_trainer_mlflow(
        X_train: np.ndarray,
        y_train: np.ndarray,
    ) -> ClassifierMixin:
        """Train a sklearn SVC classifier and log to MLflow."""
        mlflow.sklearn.autolog()  # log all model hparams and metrics to MLflow
        model = SVC(gamma=gamma)
        model.fit(X_train, y_train)
        return model

    return digits_pipeline(
        importer=importer(),
        trainer=svc_trainer_mlflow(),
        evaluator=evaluator(),
    )

Now, let's do the same for our decision tree trainer step:

In [ ]:
from sklearn.tree import DecisionTreeClassifier


def build_tree_mlflow_pipeline():
    @enable_mlflow  # setup MLflow
    @step(
        enable_cache=False
    )  # disable caching so we log **every** run to MLflow
    def tree_trainer_with_mlflow(
        X_train: np.ndarray,
        y_train: np.ndarray,
    ) -> ClassifierMixin:
        """Train a sklearn decision tree classifier and log to MLflow."""
        mlflow.sklearn.autolog()  # log all model hparams and metrics to MLflow
        model = DecisionTreeClassifier()
        model.fit(X_train, y_train)
        return model

    return digits_pipeline(
        importer=importer(),
        trainer=tree_trainer_with_mlflow(),
        evaluator=evaluator(),
    )

Finally, to run our MLflow pipelines with ZenML, we first need to add MLflow into our ZenML MLOps stack.
To do so, we first register a new experiment tracker with ZenML and then add it into our current stack.
We can then use `zenml stack describe` to show an overview of our currently active MLOps stack.

In [ ]:
# Register the MLflow experiment tracker
!zenml experiment-tracker register mlflow_tracker --type=mlflow

# Add the MLflow experiment tracker into our default stack
!zenml stack update default -e mlflow_tracker

In [ ]:
# See an overview of your current MLOps stack
!zenml stack describe

And that's it, we're all setup! Now we can simply call `pipeline.run()` for all of our pipelines to run them and automatically log all of our pipeline runs to MLflow. Let's try it out and do a few pipeline runs with different hyperparameters:

In [ ]:
for gamma in (1e-4, 1e-3, 1e-2, 1e-1):
    build_svc_mlflow_pipeline(gamma=gamma).run()
build_tree_mlflow_pipeline().run()

To compare all of our runs within the MLflow UI, run the following cell, then open http://127.0.0.1:4997/ in your browser.

In [ ]:
# This will start a serving process for mlflow
#  - if you want to continue in the notebook you need to manually
#  interrupt the kernel
from zenml.integrations.mlflow.mlflow_utils import get_tracking_uri

!mlflow ui --backend-store-uri="{get_tracking_uri()}" --port=4997

The first thing you will see is an overview of all your runs as shown below:

![MLflow UI](_assets/2-1/mlflow_ui.png)

Click on the `Parameters >` tab on top of the table to see *all* hyperparameters of your model. Now you can see at a glance which model performed best and which hyperparameters changed between different runs. In our case, we can see that the SVC model with `gamma=0.001` achieved the best validation accuracy of `0.969`.

If we click on one of the links in the `Start Time` column, we can see additional details of the respective run. In particular, under the `Artifacts` tab, we can find a `model.pkl` file, which we could now use to load our model in an inference/production environment to deploy it. In the next lesson, `2-2_Local_Deployment.ipynb`, we will learn how to do this automatically as part of our pipelines with the [MLflow Models component](https://mlflow.org/docs/latest/models.html).

## Alternative Tool: Weights & Biases

Of course, MLflow is not the only tool you can use for experiment tracking. In the following example, we will show how we could achieve the same with another experiment tracking tool: Weights & Biases.
This example assumes you already have some familiarity with W&B. In particular, you a need a Weights & Biases account, which you can set up for free [here](https://wandb.ai/login?signup=true).

You then need to define the three variables below to authorize yourself in W&B and to tell ZenML which entity/project you want to log to:
- `WANDB_API_KEY`: your API key, which you can retrieve at [https://wandb.ai/authorize](https://wandb.ai/authorize). Make sure to never share this key (in particular, make sure to remove the key before pushing this notebook to any public Git repositories!).
- `WANDB_ENTITY`: the entity (team or user) that owns the project you want to log to. If you are using W&B alone, just use your username here.
- `WANDB_PROJECT`: the name of the W&B project you want to log to. If you have never used W&B before or want to start a new project, simply type the new project name here, e.g., "zenbytes".

In [ ]:
WANDB_API_KEY = None  # TODO: replace this with your W&B API key
WANDB_ENTITY = None  # TODO: replace this with your W&B entity name
WANDB_PROJECT = "zenbytes"  # TODO: replace this with your W&B project name (if you want to log to a specific project)

In [ ]:
# Register the W&B experiment tracker
!zenml experiment-tracker register wandb_tracker --type=wandb --api_key={WANDB_API_KEY} --entity={WANDB_ENTITY} --project_name={WANDB_PROJECT}

# Create a new stack with W&B experiment tracker in it
!zenml stack register wandb_stack -m default -a default -o default -e wandb_tracker

# Set it as the active stack
!zenml stack set wandb_stack

# See an overview of your current MLOps stack
!zenml stack describe

Next, we need to add wandb to our `svc_trainer` step and use that to initialize our ZenML pipeline. The overall setup is the exact same as for the MLflow example above: we simply add an `@enable_wandb` decorator to our step and then we can use `wandb` functionality within the step as we wish.

The main difference to the MLflow example before is that W&B has no sklearn autolog functionality. Instead, we need to call `wandb.log(...)` for each value we want to log to Weights & Biases.

Since we also want to log our validation score, we need to adjust our `evaluator` step accordingly as well.

Note that, despite wandb being used in different steps within a pipeline, ZenML handles initializing wandb and ensures that the experiment name is the same as the pipeline name and that the experiment run name is the same as the pipeline run name. This establishes a lineage between pipelines in ZenML and experiments in wandb.

In [ ]:
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
import wandb
from zenml.integrations.wandb.wandb_step_decorator import enable_wandb
from zenml.steps import step, BaseStepConfig


def build_svc_wandb_pipeline(gamma=1e-3):
    @enable_wandb  # setup wandb
    @step(enable_cache=False)
    def svc_trainer_wandb(
        X_train: np.ndarray,
        y_train: np.ndarray,
    ) -> ClassifierMixin:
        """Train a sklearn SVC classifier and log to W&B."""
        wandb.log({"gamma": gamma})  # log gamma hparam to wandb
        model = SVC(gamma=gamma)
        model.fit(X_train, y_train)
        return model

    @enable_wandb  # setup wandb
    @step
    def evaluator_wandb(
        X_test: np.ndarray,
        y_test: np.ndarray,
        model: ClassifierMixin,
    ) -> float:
        """Calculate the accuracy on the test set and log to W&B."""
        test_acc = evaluator(X_test, y_test, model)
        wandb.log({"test acc": test_acc})  # log test_acc to wandb
        return test_acc

    return digits_pipeline(
        importer=importer(),
        trainer=svc_trainer_wandb(),
        evaluator=evaluator_wandb(),
    )

Finally, execute the cell below to run your pipeline with different gamma values. You should then see your runs recorded in your Weights & Biases project.

In [ ]:
for gamma in (1e-4, 1e-3, 1e-2, 1e-1):
    build_svc_wandb_pipeline(gamma=gamma).run()

Next, we will learn how to deploy models locally with MLflow in `2-2_Local_Deployment.ipynb`.